## New York City Taxi Fare Prediction
### Can you predict a rider's taxi fare?


This is one of the Kaggle Competition for biginner, alredy completed.
Even it is very simple and straitforward for estimation, i thought it will be a good example to practice estimation problem from the scrath. 

You can refer to here for details, https://www.kaggle.com/c/new-york-city-taxi-fare-prediction 

In [1]:
import os
import sys
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader

In [2]:
files = os.listdir('./')
files

['make_dataset.ipynb',
 'test_s.csv',
 'train_s.csv',
 '.DS_Store',
 'NewYork_City_Taxi_Fare_Prediction_tf_1.ipynb',
 'NewYork_City_Taxi_Fare_Prediction_tf.ipynb',
 'test.csv',
 'submission.csv',
 'GCP-Coupons-Instructions.rtf',
 'train.csv',
 '.ipynb_checkpoints',
 'sample_submission.csv',
 'NewYork_City_Taxi_Fare_Prediction.ipynb']

### Please download pre-created train_s file which samples 10% of the original file randomly
**train_s.csv file download** 
[train_s.csv](https://drive.google.com/uc?export=download&id=1AbO1jfrwJ0IKSJQactC6msPNub131LAh)
<br>**test.csv file download**
[test_s.csv](https://drive.google.com/uc?export=download&id=1iE_JilybsBIBpaTveD6vX8AfwaZzI8i8)

### Let's read csv data into pandas dataframe
It will take some time to load as train_s.csv is a bit huge, about 601MB

In [3]:
train_df = pd.read_csv('./train_s.csv', index_col=0)
test_df = pd.read_csv('./test_s.csv', index_col=0)

**Let's see what the data looks like**

In [4]:
train_df.head(2)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
key,,,,,,,
2011-07-27 19:31:00.000000138,11.7,2011-07-27 19:31:00 UTC,-73.950722,40.779295,-73.97878,40.743513,1
2012-09-02 17:31:00.000000153,8.9,2012-09-02 17:31:00 UTC,-73.992025,40.735335,-74.00752,40.717647,1


### Removing Nan and Outliers

In [5]:
# Let's look over train data if there are Nan in any of features
print(train_df.isnull().sum() , test_df.isnull().sum())

fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude    27
dropoff_latitude     27
passenger_count       0
dtype: int64 fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64


In [6]:
# Remove missing values
train_df = train_df.dropna(how = 'any')

### From [the starter code](https://www.kaggle.com/dster/nyc-taxi-fare-starter-kernel-simple-linear-model) 

Add columns

```
# Given a dataframe, add two new features 'abs_diff_longitude' and
# 'abs_diff_latitude' reprensenting the "Manhattan vector" from
# the pickup location to the dropoff location.
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(train_df)
```

In [7]:
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

# add longitude, latitude difference to column features
add_travel_vector_features(train_df)
add_travel_vector_features(test_df)

### Filtering out outliers
From the starter code<br>
"We expect most of these values to be very small (likely between 0 and 1) since it should all be differences between GPS coordinates within one city. For reference, one degree of latitude is about 69 miles. However, we can see the dataset has extreme values which do not make sense. Let's remove those values from our training set. Based on the scatterplot, it looks like we can safely exclude values above 5"

In [8]:
train_df = train_df[(train_df.abs_diff_longitude < 5.0) & (train_df.abs_diff_latitude < 5.0)]

In [9]:
## Let's see the size of data again now
print('The size of train data =', len(train_df))
print('The size of test =', len(test_df))
print('The types of data =', train_df.dtypes)

The size of train data = 4977770
The size of test = 493647
The types of data = fare_amount           float64
pickup_datetime        object
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
passenger_count         int64
abs_diff_longitude    float64
abs_diff_latitude     float64
dtype: object


### Let's implement your own pytorch code here